In [1]:
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-5
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99

#???
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

#Импорт таблиц номинального режима и частичной нагрузки
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#???
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

it=50
for k in range(it):
    ######################################################################
    #Связвка высокого давления
    start_time = time.time()
    for j in range(it):
        for i in range(it):
            PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',       KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
            water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
            IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
            water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
            water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
            Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
            Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
            print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;
        EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
        water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]

        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
        Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
    print("ВД:--- %s сек. ---" % round((time.time() - start_time)))
    ######################################################################



    ######################################################################
    #Связка низкого давления
    it=50
    for j in range(it):
        #Связка ППНД+ИНД
        for i in range(it):
            #Расчёт ППНД
            PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
            water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
            #Расчёт ИНД
            IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
            water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
            #Переопределение расхода пара
            water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
            Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
            Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
            print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;
        #ПЭН
        water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
        PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
        water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]


        #Расчёт расхода в ГПК
        water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
        water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
        #Расчёт ГПК
        GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
        water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]

        Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
        Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД+ГПК',abs((Qgas1-Qwat1)/Qgas1))
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
    print("НД:--- %s сек. ---" % round((time.time() - start_time)))
    ######################################################################
    Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
    Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
    
    
    
    print('dQ/Qsumm',abs((Qgasall-Qwatall)/Qgasall))
    if  abs((Qgasall-Qwatall)/Qgasall) < calctolerance:
        break;


dQ/Q ПЕВД+ИВД 0.6347799607161102
dQ/Q ПЕВД+ИВД -0.387708257795798
dQ/Q ПЕВД+ИВД 0.04500246877153103
dQ/Q ПЕВД+ИВД -0.017088670740956886
dQ/Q ПЕВД+ИВД 0.006161094774647087
dQ/Q ПЕВД+ИВД -0.0022642062423093136
dQ/Q ПЕВД+ИВД 0.0008263164057464336
dQ/Q ПЕВД+ИВД -0.00030233270322600395
dQ/Q ПЕВД+ИВД 0.00011051438443262539
dQ/Q ПЕВД+ИВД -4.041109520700329e-05
dQ/Q ПЕВД+ИВД 1.4775026715079174e-05
dQ/Q ПЕВД+ИВД -5.402263099137137e-06
dQ/Q ПЕВД+ИВД+ЭВД -0.0022922415325220395
dQ/Q ПЕВД+ИВД -0.0014316245931800895
dQ/Q ПЕВД+ИВД 0.0005206939048709259
dQ/Q ПЕВД+ИВД -0.00018968603105162695
dQ/Q ПЕВД+ИВД 6.906111057327747e-05
dQ/Q ПЕВД+ИВД -2.5149218993864224e-05
dQ/Q ПЕВД+ИВД 9.15760020996857e-06
dQ/Q ПЕВД+ИВД+ЭВД -0.0019138084090542433
dQ/Q ПЕВД+ИВД -0.0012098414705709192
dQ/Q ПЕВД+ИВД 0.000438607668157959
dQ/Q ПЕВД+ИВД -0.00015922603486393907
dQ/Q ПЕВД+ИВД 5.777471937393842e-05
dQ/Q ПЕВД+ИВД -2.096714555380997e-05
dQ/Q ПЕВД+ИВД 7.608737112767072e-06
dQ/Q ПЕВД+ИВД+ЭВД -0.0016021878302366595
dQ/Q ПЕВ

dQ/Q ППНД+ИНД 4.3256004319585435e-07
dQ/Q ППНД+ИНД+ГПК 0.0005719984002261981
dQ/Q ППНД+ИНД -1.1976775475951735e-05
dQ/Q ППНД+ИНД 2.9875891220140605e-07
dQ/Q ППНД+ИНД+ГПК 0.00039719680874985235
dQ/Q ППНД+ИНД -8.284990479214957e-06
dQ/Q ППНД+ИНД+ГПК 0.0002799808109984673
dQ/Q ППНД+ИНД -5.5235086810200435e-06
dQ/Q ППНД+ИНД+ГПК 0.00019416320396039086
dQ/Q ППНД+ИНД -3.83102612777232e-06
dQ/Q ППНД+ИНД+ГПК 0.00013469915576611862
dQ/Q ППНД+ИНД -2.654252391407042e-06
dQ/Q ППНД+ИНД+ГПК 9.343122883093672e-05
dQ/Q ППНД+ИНД -1.8394816322563441e-06
dQ/Q ППНД+ИНД+ГПК 6.480025443429006e-05
dQ/Q ППНД+ИНД -1.2750285792428082e-06
dQ/Q ППНД+ИНД+ГПК 4.493985151847328e-05
dQ/Q ППНД+ИНД -8.838822460837985e-07
dQ/Q ППНД+ИНД+ГПК 3.116491528231153e-05
dQ/Q ППНД+ИНД -6.127786039398012e-07
dQ/Q ППНД+ИНД+ГПК 2.1611553239549083e-05
dQ/Q ППНД+ИНД -4.2485119612073046e-07
dQ/Q ППНД+ИНД+ГПК 1.4986358070044493e-05
dQ/Q ППНД+ИНД -2.9456883139211407e-07
dQ/Q ППНД+ИНД+ГПК 1.039200531306978e-05
dQ/Q ППНД+ИНД -2.042435336575

C:\Users\Admin\Documents\GitHub\GZ\cotel.py:72: RuntimeWarning: invalid value encountered in double_scalars
  kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\scipy\optimize\_minpack_py.py:237: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
Error in sys.excepthook:
Traceback (most recent call last):
  File "c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py", line 1958, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py", line

error: Result from function call is not a proper array of floats.

In [ ]:
gas_streams

In [ ]:
water_streams